<a href="https://colab.research.google.com/github/ByungjunKim/ModernKoreanSubword/blob/main/ModernKoreanSubword_Train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 근대 한국어 서브워드 형태소 분석기 학습

### 서브워드 학습 준비

In [52]:
import pandas as pd

# kiwi 로드
from kiwipiepy import Kiwi
kiwi = Kiwi()
from kiwipiepy.sw_tokenizer import SwTokenizer
from kiwipiepy.sw_tokenizer import SwTokenizerConfig
import gc
import hanja
import pickle

In [2]:
# 근대 잡지
mag = pd.read_pickle('./한국근현대잡지자료.pickle')
mag

,level3id,mainTitle_x,subjectClass,pTitle,paragraph,level2id,level1id,mainTitle_y,publisher_x,dateIssued_x,mainTitle,publisher_y,dateIssued_y
0,ma_061_0010_0010,卷頭一聲,,卷頭一聲,\n卷頭一聲\n又影生\n,ma_061_0010,ma_061,공제 제1호,,1920-09-10,공제,,1920-09-10
1,ma_061_0010_0020,創刊辭,,創刊辭,\n創刊辭\n,ma_061_0010,ma_061,공제 제1호,,1920-09-10,공제,,1920-09-10
2,ma_061_0010_0030,勞働萬能論,,勞働萬能論,\n勞働萬能論\n趙誠惇\n,ma_061_0010,ma_061,공제 제1호,,1920-09-10,공제,,1920-09-10
3,ma_061_0010_0040,勞働化하라,,勞働化하라,\n勞働化하라\n南相協\n,ma_061_0010,ma_061,공제 제1호,,1920-09-10,공제,,1920-09-10
4,ma_061_0010_0050,勞働問題는 社會의 根本問題이라,,勞働問題는 社會의 根本問題이라,\n勞働問題는 社會의 根本問題이라\n金明植\n,ma_061_0010,ma_061,공제 제1호,,1920-09-10,공제,,1920-09-10
...,...,...,...,...,...,...,...,...,...,...,...,...,...
588569,ma_089_0040_0110,쏘위엣트 露西亞의 붉은 新聞,,쏘위엣트 露西亞의 붉은 新聞,\n쏘위엣트 露西亞의 붉은 新聞\n金世鎔\n,ma_089_0040,ma_089,철필 제2권 제1호,,1931-01-11,철필,,1930-07-09
588570,ma_089_0040_0120,獨逸及英國 新聞界 巡訪印象,,獨逸及英國 新聞界 巡訪印象,\n獨逸及英國 新聞界 巡訪印象\n金賢準\n,ma_089_0040,ma_089,철필 제2권 제1호,,1931-01-11,철필,,1930-07-09
588571,ma_089_0040_0130,三大新聞社 人材巡禮,,三大新聞社 人材巡禮,\n三大新聞社 人材巡禮\n覆面記者\n,ma_089_0040,ma_089,철필 제2권 제1호,,1931-01-11,철필,,1930-07-09
588572,ma_089_0040_0140,新聞語辭典,,新聞語辭典,\n新聞語辭典\n編輯部\n,ma_089_0040,ma_089,철필 제2권 제1호,,1931-01-11,철필,,1930-07-09


In [3]:
mag_paragraph = mag['paragraph'].tolist()

In [4]:
del mag

In [5]:
chosun = pd.read_parquet('/mnt/hdd/bjkim/ModernConcept/chosun_preprocessed.parquet',columns=['year','type','body_archaic'])

In [8]:
chosun = chosun.dropna()
chosun

,year,type,body_archaic
0,1920,smallad,祝 …朝；蘚 n 報創 刊\n趙… . … . 命· 九\n
4,1920,article,朝鮮日報\n創刊을祝함\n候爵 大隈重信\n朝鮮合邦以來歲를閱한지玆에十年이되鮮人의自營하난...
5,1920,article,朝鮮現下의\n工業狀態\n를鑑하야振興策\n을一二를言함\n朝鮮의工業으로言하면從來此를賤視...
6,1920,etc,祝詞와祝電\n
7,1920,article,"大正八年朝鮮貿易槪况(一)\n(朝鮮總督府財務局調査)\n一,總况\n大正八年朝鮮貿易額은輸..."
...,...,...,...
4679014,1999,article,폐품 할머니\n40년간 고물 수집\n외대 1억 기증후 타계\n 지난 7월 장경자(張...
4679015,1999,article,고시준비생 장학금\n외대에 3억원 전달\n商街주인 할머니\n 지난 14일힐튼호텔 컨...
4679016,1999,article,“1900년에 출두하시오”\n수원지법 컴퓨터‘Y2K’\n피고인에 엉뚱한 소환장\n ...
4679017,1999,photo,◇故장경자씨\n


In [9]:
chosun = chosun[chosun['year']<=1940].reset_index(drop=True)
chosun

,year,type,body_archaic
0,1920,smallad,祝 …朝；蘚 n 報創 刊\n趙… . … . 命· 九\n
1,1920,article,朝鮮日報\n創刊을祝함\n候爵 大隈重信\n朝鮮合邦以來歲를閱한지玆에十年이되鮮人의自營하난...
2,1920,article,朝鮮現下의\n工業狀態\n를鑑하야振興策\n을一二를言함\n朝鮮의工業으로言하면從來此를賤視...
3,1920,etc,祝詞와祝電\n
4,1920,article,"大正八年朝鮮貿易槪况(一)\n(朝鮮總督府財務局調査)\n一,總况\n大正八年朝鮮貿易額은輸..."
...,...,...,...
930051,1940,article,"各界人物招致\n首相,意見交換豫定\n【東京電話同盟】新體制確立에關하여 그根本原理組織方法..."
930052,1940,article,臨時家族手當\n十月부터實施决定\n【東京電話同盟】政府는 九日閣議에서 下級官吏에 臨時家...
930053,1940,article,"外相,首相과會談\n【東京電話同盟】松岡外相은 九日定例閣議散會後 首相官邸에남어 近衞首相..."
930054,1940,article,人事·集會\n▲朱□宜氏(前普成中學校長)退任人事次十日來社\n▲鈴木文次郞氏(全朝鮮洋服商...


In [10]:
# br 태그 삭제
chosun['body_archaic'] = chosun['body_archaic'].str.replace('</br>','')

In [11]:
# 【三十日東京特電】 처럼 특파원 삭제
chosun['body_archaic'] = chosun['body_archaic'].str.replace('\【.+\】','',regex=True)

In [12]:
chosun = chosun.dropna(subset=['body_archaic']).reset_index(drop=True)

In [14]:
chosun_paragraph = chosun['body_archaic'].tolist()

In [15]:
del chosun

In [18]:
donga = pd.read_parquet('/mnt/hdd/bjkim/ModernConcept/donga_preprocessed.parquet',columns=['year','body_ori'])
donga

,year,body_ori
0,1920,主旨를宣明하노라\n蒼天에太陽이빗나고大地에淸風이불도다山靜水流하며草木昌茂하며百花爛發하며...
1,1920,九十度난太甚\n혹독한형벌인우에\n따리는데도불공도\n스코필트씨밀\n관텽에서 허가하는 ...
2,1920,肉體보다도精神\n몸을앗김보다\n명예를위하라\n방하춍독무의원장의말사월초하ㄹ날로무터 태...
3,1920,四十七人의公判\n셰길이넘는긔록을조사하기에\n판검사아홉명이매여달녀잇다\n느저도금년안에...
4,1920,姜宇奎公判은\n금월오일개정\n남대문역두에서 재등춍독에게폭탄을던지여 한세상을 떠들니든...
...,...,...
3125897,1999,키노코너\n독립영화협의회가 주최하는 제26회 독립영화 워크숍이 2000년 1월3일부...
3125898,1999,한국영화 대약진“올해만 같아라”\n점유율 40% 육박…작품성은 미흡\n외화 상대적 ...
3125899,1999,1999년 영화계 결산\n99 한국영화 흥행 베스트10\n99 외국영화 흥행 베스트10
3125900,1999,"Y2K D-1\n업무 본격시작 1월3,4일 더 위험\n‘Y2K D—1’\n컴퓨터의 ..."


In [19]:
donga = donga[donga['year']<=1940].dropna().reset_index(drop=True)
donga

,year,body_ori
0,1920,主旨를宣明하노라\n蒼天에太陽이빗나고大地에淸風이불도다山靜水流하며草木昌茂하며百花爛發하며...
1,1920,九十度난太甚\n혹독한형벌인우에\n따리는데도불공도\n스코필트씨밀\n관텽에서 허가하는 ...
2,1920,肉體보다도精神\n몸을앗김보다\n명예를위하라\n방하춍독무의원장의말사월초하ㄹ날로무터 태...
3,1920,四十七人의公判\n셰길이넘는긔록을조사하기에\n판검사아홉명이매여달녀잇다\n느저도금년안에...
4,1920,姜宇奎公判은\n금월오일개정\n남대문역두에서 재등춍독에게폭탄을던지여 한세상을 떠들니든...
...,...,...
1001312,1940,手術界의驚異\n醫學朝鮮에凱旋歌\n없는男子生殖 를붙이기에 드디어成功\n世專李鶴松博士의...
1001313,1940,三年後에오는大京城\n二百萬人口를收容\n十八年度 지七個地區에新市街地를形成\n京城府에서...
1001314,1940,張皷峯事件二 年記念\n今日、嚴肅히擧行\n正午를期해一齊默禱\n황군전사(皇軍戰史)에 최...
1001315,1940,辯護士試驗延長\n受驗生代表들이再次當局에陳情\n누보한바와같이 조선에잇어서의 변호사시험...


In [20]:
donga_paragraph = donga['body_ori'].tolist()

In [21]:
del donga

In [22]:
paragraph_list = mag_paragraph + chosun_paragraph + donga_paragraph
len(paragraph_list)

2519947

In [23]:
del mag_paragraph, chosun_paragraph, donga_paragraph

In [27]:
gc.collect()

764

In [50]:
len(paragraph_list)

2519947

In [49]:
len(list(dict.fromkeys(paragraph_list)))

2313177

In [51]:
paragraph_list = list(dict.fromkeys(paragraph_list))

In [53]:
# save
with open('paragraph_list.pickle', 'wb') as f:
    pickle.dump(paragraph_list, f, pickle.HIGHEST_PROTOCOL)

### 서브워드 학습

In [54]:
# https://bab2min.github.io/kiwipiepy/v0.15.2/kr/sw_tokenizer.html#kiwipiepy.sw_tokenizer.SwTokenizerConfig
config = SwTokenizerConfig(split_chinese=False) # split_chinese 제외하고 모두 기본 옵션 사용

In [56]:
# 학습
# save_path = 토크나이저.json 파일 저장 위치
# texts = 학습할 데이터(리스트)
# vocab_size = 단어수
# config = 학습 옵션
tokenizer = SwTokenizer.train(save_path='./2410131_vo64000_tokenizer.json', texts=paragraph_list, vocab_size=64000, config=config)

Reducing #1:   0%|          | 0/1000 [00:00<?, ?it/s, vocab_size=8564346, loss=8.72]

Iteration: 0 VocabSize: 8564346 Loss: 8.7218


Reducing #1:   0%|          | 1/1000 [00:32<8:53:59, 32.07s/it, vocab_size=6160404, loss=8.58]

Iteration: 1 VocabSize: 6160404 Loss: 8.5842


Reducing #1:   0%|          | 2/1000 [01:02<8:34:42, 30.94s/it, vocab_size=5.5e+6, loss=8.58] 

...

Iteration: 74 VocabSize: 64038 Loss: 7.9781


Reducing #1:   8%|▊         | 75/1000 [37:20<7:11:17, 27.98s/it, vocab_size=64002, loss=7.98]

Iteration: 75 VocabSize: 64002 Loss: 7.9781


Reducing #1:   8%|▊         | 76/1000 [37:48<7:39:38, 29.85s/it, vocab_size=64000, loss=7.98]


Iteration: 76 VocabSize: 64000 Loss: 7.9781
Finished. Iteration: 76 VocabSize: 64000 Loss: 7.9781


In [57]:
paragraph_list[20000]

' 2. 耕地面積   全地面積의 百分之15%'

In [58]:
hanja.translate(paragraph_list[20000], 'substitution')

' 2. 경지면적   전지면적의 백분지15%'

In [59]:
# 서브워드 토크나이저 성능 확인
print([tokenizer.id2vocab[i] for i in tokenizer.encode(paragraph_list[20000])])

['2./S', '##耕地面積', '全', '##地', '##面積', '의/J', '百分之', '##15', '%']


In [60]:
[item for item in paragraph_list if '新郞' in item]

['新郞될 사람 「그러타면 그 寫眞을 좀 보여 주세요.」',
 '新郞 될 사람 「아니 아니 그것보다도 그 아버지가 經營한다는 會社 寫眞말이얘요.」',
 '\n新郞新婦의 新婚共同日記\n',
 '1931年 1月 1日 快晴(記者註=氏는 新進小說家로서 최근 結婚한 新郞되시는 분이다.)',
 ' 新郞 「우리 둘이 이제는 결혼햇스니 어듸 당신 흉점을 몃가지 말해볼가요.」',
 '조선에서 가장 존경을 밧는 분도 春園이요, 가장 미움을 밧는 이도 春園이다. 그가 新郞이 된다면 祝辭드릴 사람도 數萬일 것이요. 그 結婚式에 돌 던질 이도 數萬일 것이며, 또 그가 不幸히 저 세상에 간다 하면 그 葬式에 吊花를 보내며 눈물 뿌릴 사람도 數萬일 것이요. 잘 죽엇다고 욕할 사람도 수만이리라.',
 '\n落馬한 新郞\n',
 '安氏가 大邱서 靑年團 首領級으로 梧桐옷을 입을 때 黃氏 亦同根異枝인 婦人團事件으로 梧桐衣人이 되엇다. 이제 往事는 割愛하고, 米州로부터 도라온 女史는 老靑春에 新郞을 맛어 人生의 잔채床에 마조 안즌 뒤 다시 事業人으로서 女子基督靑年會를 만들어 가지고 包야 將이야 하고 楚漢을 다트고 잇더니 最近에는 朴熙道氏 失脚以后 中央保育學校의 經營을 마터 지금은 동교 교장으로 幼稚園 할머니(保姆가 아히들 어머니요 그 保姆를 낫는 어머니는 할머니일 밧게)노릇하기에 朝夕 눈코 못뜬다.',
 ' 예루살넴의 多角形 穹窿寺院을 背景하고 新婦 마리아와 新郞 요셉이 對面해서 잇고 그 가온대 大僧이 서서 두 사람의 손을 쥐게하며 요셉이 妻女에 손에 반지를 끼우랴고 하고 잇다. 마리아를 擁護해잇는 여러 妻女들은 美望의 表情이 充滿하고 요셉편에 잇는 拒絶을 當한 求婚靑年들은 絶望의 苦痛에 못견대서 막대기를 꺽는 者 참아 볼 수 업서 고개를 도리키는 者가 잇다. 여러 가지(枝)는 부즈러지고 한가지만 요셉으로 인하야 꼿이 피엿다. 이와 갓치 요셉과 마리아는 公衆에 압헤서 法律上 正式 結婚式을 擧行하는 것이다.',
 ' 10分! 20分-. 깃븜과 행복이 춤출 新房이 답답함과 염려스러움에 잠기엇다. 까

In [ ]:
print([tokenizer.id2vocab[i] for i in tokenizer.encode('新郞될 사람 「그러타면 그 寫眞을 좀 보여 주세요.')])

['新', '##郞', '되/V', 'ᆯ/E', '사람', '「', '그러/V', '어/E', '타/V', '면/E', '그', '寫眞', '을/J', '좀', '보이/V', '어/E', '주/V', '세요/E', '.']
